# Cython: A First Look

## Some Jupyter lab notes:
* Jupyter lab let's us make cells and run code in a nicely formatted way
* We also can use things like magic cells - these allow us to do special operations on code
* Rerunning cells is super easy
* For Cython - the notebook abstracts all of the compilation away
* Also for Cython - allows you to profile your code

## Typical sieve algorithm:

1. Create a list of integers 2 -> N
2. Start at 2, all factors of it are marked in the list as non-prime (false)
3. Go to next true index
4. Mark all factors of it in the list as false
5. Go to step 3
6. All remaining true indices are prime numbers

Here's a basic sieve implementation. Nothing special.

Might not even be the most efficient!

In [1]:
def sieve(sieve_length):
    sieve_table = [True for x in range(sieve_length)]
    sieve_table[0] = False
    sieve_table[1] = False
    
    for i in range(2,int(sieve_length**0.5)+1):
        if sieve_table[i]:
            for marker in range(i*i, sieve_length, i):
                sieve_table[marker] = False
    
    return [i for i, t in enumerate(sieve_table) if t]

Testing base functionality:

In [2]:
primes = sieve(1_000)
print(','.join([str(p) for p in primes]))

2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,571,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,661,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,883,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997


Everything appears to be working, but how fast is it?

Time for some basic benchmarking!

In [3]:
%timeit sieve(1_000_000)

109 ms ± 5.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
%timeit sieve(10_000_000)

1.39 s ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Anecdotally - I happen to know this is pretty slow.

## First steps into Cython

In [5]:
%load_ext Cython

In [6]:
%%cython

def sieve_magic(sieve_length):
    sieve_table = [True for x in range(sieve_length)]
    sieve_table[0] = False
    sieve_table[1] = False
    
    for i in range(2,int(sieve_length**0.5)+1):
        if sieve_table[i]:
            for marker in range(i*i, sieve_length, i):
                sieve_table[marker] = False
    
    return [i for i, t in enumerate(sieve_table) if t]

In [7]:
primes_magic = sieve_magic(1_000)
print(','.join([str(p) for p in primes_magic]))

2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101,103,107,109,113,127,131,137,139,149,151,157,163,167,173,179,181,191,193,197,199,211,223,227,229,233,239,241,251,257,263,269,271,277,281,283,293,307,311,313,317,331,337,347,349,353,359,367,373,379,383,389,397,401,409,419,421,431,433,439,443,449,457,461,463,467,479,487,491,499,503,509,521,523,541,547,557,563,569,571,577,587,593,599,601,607,613,617,619,631,641,643,647,653,659,661,673,677,683,691,701,709,719,727,733,739,743,751,757,761,769,773,787,797,809,811,821,823,827,829,839,853,857,859,863,877,881,883,887,907,911,919,929,937,941,947,953,967,971,977,983,991,997


In [8]:
%timeit sieve_magic(1_000_000)

66 ms ± 4.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Exploring with Cython

Cython gives us the ability to view how our code has compiled!

Let's try it:

# Splitting things up

It looks like working on these list comprehensions is going to be a struggle... Let's split some things up.


# Calling STL Functions

At this point we know that there's more we can do with that inner for loop - but let's have a look at the list access that's being done.

Why don't we replace it with a C++ structure?

In [9]:
%reload_ext Cython

In [10]:
%%cython
# distutils: language=c++

import cython

from libcpp.vector cimport vector

def do_stuff():
    cdef vector[int] totally_a_list
    totally_a_list.push_back(100)
    return totally_a_list[0]

In [11]:
do_stuff()

100

that was easy! Let's rewrite our previous code now.

# Battling the Inner Loop

There's other smaller optimizations to do for sure - but what about that inner for loop?